In [213]:
import numpy as np
import opensimplex
import random
from time import sleep
import scipy.linalg as sla


In [275]:
#     1)number of stores 
#     2)number of products :- an int representing the number of products
#     3timeframe :- an int representing the number of days we simulate
#     4)factory_capacity :- an int representing how many total products the factory can produce in a day
#     5)DC_capacity :- an int representing how many total products can be stored:
#     6)demand range min :- an int saying the least value of each products order
#     7)demand range max :- an int saying the max value of each products order
#     8)truck_capacity :- total products truck can carry
#     9)data = array of shape (stores, products, demand_range, timeframe) where data[w][x][y][z] = probability of
#         store w wanting product x in (y + demand_range_min) amount on day z 
#     10)truck_max_cap :- hypothetically, the number of trucks on a day can be min(max_supply, max_demand)
#         i.e. 1 truck per product from demand to supply 
#         max_supply = DC_capacity
#         max_demand = demand_range_max * number of stores * number of products
#         but simulating this wastes a lot of space, so we resonably cap the number of trucks to save space
#     11)DC_cap_to_noise_mean_factor: -I generate 4d simplex noise and multiply that with DC_Capacity to get 
#             data array, but if demand always tends towards max supply then because of inoptimality 
#         it will keep on cummilating, keeping this near 1 is like a stress test for the algorithm

class TestCase:
    def __init__(self, number_of_stores,number_of_products, timeframe = 90, factory_capacity = 200, 
                       DC_capacity = 50, demand_range_min = 10, demand_range_max = 400,
                      truck_capacity = 20, truck_max_cap = 0.1):
        self.number_of_stores = number_of_stores
        self.number_of_products = number_of_products
        self.timeframe = timeframe
        self.factory_capacity = factory_capacity
        self.DC_capacity = DC_capacity
        self.demand_range_min = demand_range_min
        self.demand_range_max = demand_range_max
        self.truck_capacity = truck_capacity
        self.truck_max_cap = truck_max_cap

        self.maximum_truck = truck_max_cap * min(DC_capacity, demand_range_max * number_of_stores*number_of_products )
    
    def getData(self, DC_cap_to_noise_mean_factor = 1, seed = None, simplex_reduction_factor = 0.3,
                expected_value_to_distribution = "normal random distribution" ):
#         simplex_reduction_factor :- making this value low will reduce variance and all data will look the same
#                                    keeping it high will make the data look completely random 
        demand_range = self.demand_range_max - self.demand_range_min + 1
        data = np.zeros((self.number_of_stores, self.number_of_products, self.timeframe, demand_range), 
                        dtype=np.float32)
        if seed == None:
            opensimplex.random_seed()
        else:
            opensimplex.seed(seed)
        for store in np.arange(self.number_of_stores):
            for product in np.arange(self.number_of_products):
                for time in np.arange(self.timeframe):
                    data[store, product, time, 0] = opensimplex.noise3(store*simplex_reduction_factor, 
                                                                       product*simplex_reduction_factor, 
                                                                       time*simplex_reduction_factor)
        #normalising from -1,1 -> 0, 1
        data[:, :, :, 0] += 1
        data[:, :, :, 0] /= 2
        
        total = np.sum(data[:,:,:,0])
        cumm_dmnd_ = self.timeframe * self.DC_capacity * DC_cap_to_noise_mean_factor
        data[:, :, :, 0] *= cumm_dmnd_ / total 
        def normal_random_distribution(data_array, X_array, expd_vl):
            def recurse(left_i, rght_i, E, P):
                if left_i + 3 <= rght_i:
                    middle = left_i + np.argmax( X_array[ left_i : rght_i ] > E ) - 1
                    if middle + 2 == rght_i:
                        return recurse(middle, rght_i, E, P)
                    if left_i == middle:
                        return recurse(left_i, left_i + 2, E, P)
                    p_l = random.uniform(0, P)
                    p_r = P - p_l
                    x_l_upper = min((E - X_array[ middle + 1 ] * p_r * p_r) / p_l, X_array[ middle ] * p_l)
                    x_l_lower = max((E - X_array[ rght_i - 1 ] * p_r * p_r) / p_l, X_array[ left_i ] * p_l)
                    x_l = random.uniform(x_l_lower, x_l_lower)
                    x_r = (E - p_l * x_l) / p_r
                    recurse(left_i, middle +1, x_l, p_l)
                    recurse(middle +1, rght_i, x_r, p_r)
                elif left_i + 2 == rght_i:
                    x_l = X_array[ left_i ]
                    x_r = X_array[ rght_i ]
                    data_array[ left_i ] = (E - P * x_r) / (x_l - x_r)
                    data_array[ rght_i ] = (E - P * x_l) / (x_r - x_l)
                elif left_i + 1 == rght_i:
                    assert False, "Algorithm is broken, this should not have happened"
                else:
                    return 
            data_len = data_array.size
            data_array[:] = 0
            X_array = np.sort(X_array)
            assert X_array[0] <=  expd_vl and expd_vl < X_array[-1], "Not possible to generate distribution"
            recurse(0, data_len, expd_vl, 1)
            print(data_array)
                
                
                
        if not callable(expected_value_to_distribution):
            if expected_value_to_distribution == "normal random distribution":
                expected_value_to_distribution = normal_random_distribution
        
        for store in np.arange(self.number_of_stores):
            for product in np.arange(self.number_of_products):
                for time in np.arange(self.timeframe):
                    expd_vl = data[store, product, time, 0]
                    expected_value_to_distribution(data[store, product, time], 
                                                   np.arange(self.demand_range_min, self.demand_range_max + 1),
                                                   expd_vl)
                    return data
#                     print(np.sum(data[store, product, time]))
        return data
tc = TestCase(2,2, DC_capacity=200)
hlpr = tc.getData(DC_cap_to_noise_mean_factor = 1, simplex_reduction_factor = 0.3)

/tmp/ipykernel_224907/906318192.py:70: RuntimeWarning: overflow encountered in double_scalars
  x_l_upper = min((E - X_array[ middle + 1 ] * p_r * p_r) / p_l, X_array[ middle ] * p_l)
/tmp/ipykernel_224907/906318192.py:71: RuntimeWarning: overflow encountered in double_scalars
  x_l_lower = max((E - X_array[ rght_i - 1 ] * p_r * p_r) / p_l, X_array[ left_i ] * p_l)
/tmp/ipykernel_224907/906318192.py:73: RuntimeWarning: overflow encountered in double_scalars
  x_r = (E - p_l * x_l) / p_r


RecursionError: maximum recursion depth exceeded while calling a Python object

In [237]:
tc = TestCase(2,2, DC_capacity=200)
hlpr = tc.getData(DC_cap_to_noise_mean_factor = 1, simplex_reduction_factor = 0.3)
# print(np.sum(hlpr < 25)*100 / hlpr.size) 
# hlpr

(391,) (391, 389) (389, 391)
[ 8.67681500e-03  8.64543413e-03  8.61405325e-03  8.58267238e-03
  8.55129151e-03  8.51991063e-03  8.48852976e-03  8.45714889e-03
  8.42576801e-03  8.39438714e-03  8.36300627e-03  8.33162539e-03
  8.30024452e-03  8.26886365e-03  8.23748278e-03  8.20610190e-03
  8.17472103e-03  8.14334016e-03  8.11195928e-03  8.08057841e-03
  8.04919754e-03  8.01781666e-03  7.98643579e-03  7.95505492e-03
  7.92367405e-03  7.89229317e-03  7.86091230e-03  7.82953143e-03
  7.79815055e-03  7.76676968e-03  7.73538881e-03  7.70400793e-03
  7.67262706e-03  7.64124619e-03  7.60986532e-03  7.57848444e-03
  7.54710357e-03  7.51572270e-03  7.48434182e-03  7.45296095e-03
  7.42158008e-03  7.39019920e-03  7.35881833e-03  7.32743746e-03
  7.29605658e-03  7.26467571e-03  7.23329484e-03  7.20191397e-03
  7.17053309e-03  7.13915222e-03  7.10777135e-03  7.07639047e-03
  7.04500960e-03  7.01362873e-03  6.98224785e-03  6.95086698e-03
  6.91948611e-03  6.88810524e-03  6.85672436e-03  6.82534349e

In [218]:
np.arange(10,20, dtype=np.int32)
random.uniform(20,10)
# 5//2
np.array([1,2,3,4]) * np.array([1,2,3,4])
E=np.array(
    [
        [-2,-4,1,-9,0],
        [3,6,0,12,3],
        [1,2,1,3,1],
        [-5,-10,3,-23,1]
    ]
)
sla.null_space(E)
# E.shape

array([[-5.19961922e-01, -8.07908734e-01],
       [ 7.73250721e-01, -3.07231209e-01],
       [-2.56634880e-01,  3.55592788e-01],
       [-2.56634880e-01,  3.55592788e-01],
       [ 1.33514196e-16,  2.67291969e-16]])

In [262]:
np.arange(-10, 10)[-1]

9